In [1]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen 
from bs4 import BeautifulSoup,Comment
import json
import requests

Scraping rookie year stats from basketball refrence, since we have compiled a set of data from 1985-2019, we will have to fidn the query names from this list and isolate those players.

In [2]:
def get_player_name_and_link(row):
    links = row.findAll('a')
    if links:
        return [links[0].getText(),"https://www.basketball-reference.com" + links[0].get('href')]

In [3]:
from string import ascii_lowercase
def get_player_names_and_links():
    player_names_and_links = []
    for first_letter in ascii_lowercase:
        if first_letter == 'x':
            continue
        url = 'https://www.basketball-reference.com/players/'+ first_letter + '/'
        html = urlopen(url)
        soup = BeautifulSoup(html,'lxml')
        rows = soup.findAll('tr')[1:]
        player_names_and_links.extend([get_player_name_and_link(row) for row in rows if int(row.findAll('td')[0].getText())>1983])
    return player_names_and_links

In [4]:
player_names_and_links = get_player_names_and_links()

In [5]:
len(player_names_and_links)

2691

In [6]:
names_of_all= [i[0] for i in player_names_and_links]

In [7]:
links_of_all = [i[1] for i in player_names_and_links]

In [8]:
print(len(names_of_all))
print(len(links_of_all))

2691
2691


In [9]:
def get_stats(hyperlink):
    url= hyperlink
    html = urlopen(url)
    soup = BeautifulSoup(html,'lxml')
    
    if not soup.findAll('tr', limit=2):
        return pd.DataFrame()
    headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
    headers = headers[1:]
    
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    if stats.shape[0]>5:
        stats= stats.head()
    num_stats= list(stats.columns.values)[4:]
    #converting numerical stat values to floats
    for stat in num_stats:
        stats[stat]= pd.to_numeric(stats[stat], errors='coerce')
    return stats.drop_duplicates().dropna(how='all')

In [10]:
def get_advanced_stats(hyperlink):
    res = requests.get(hyperlink,headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(res.text, 'lxml')
    keep=[]
    for comment in soup.find_all(string=lambda text:isinstance(text,Comment)):
        data = BeautifulSoup(comment,"lxml")
        for items in data.select("table.row_summable tr"):
            tds = [item.get_text(strip=True) for item in items.select("th,td")]
            keep.append(tds)
    adv= ['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', '', 'OWS', 'DWS', 'WS', 'WS/48', '', 'OBPM', 'DBPM', 'BPM', 'VORP']
    adv_df= pd.DataFrame(columns=adv,index=['S1','S2','S3','S4','S5'])
    if not keep:
        return pd.DataFrame()
    for i in range(len(keep)):
        if (np.array_equal(keep[i],adv)):
            adv_df.loc['S1']= keep[i+1]
            if(keep[i+2][0]!='Career'):
                adv_df.loc['S2']= keep[i+2]
                if(keep[i+3][0]!='Career'):
                    adv_df.loc['S3']= keep[i+3]
                    if(keep[i+4][0]!='Career'):
                        adv_df.loc['S4']= keep[i+4]
                        if(keep[i+5][0]!='Career'):
                            adv_df.loc['S5']= keep[i+5]
            if True:
                break
     
    adv_df= adv_df.dropna()
    return adv_df

In [11]:
a= links_of_all[1466]

In [12]:
print(a)

https://www.basketball-reference.com/players/l/lucasjo02.html


In [13]:
get_stats(a)

,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,23,HOU,NBA,PG,13.0,0.0,8.2,1.1,2.8,0.389,...,NaN,0.2,0.2,0.4,0.9,0.4,0.0,0.5,0.5,2.3
1,24,HOU,NBA,PG,47.0,0.0,8.1,1.3,3.3,0.397,...,0.789,0.2,0.6,0.8,0.7,0.4,0.0,0.3,0.8,3.3
2,2007-08,25,Did Not Play (other pro league—Italy),None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-09,26,"Did Not Play (other pro league—D-League, Spain)",None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-10,27,Did Not Play (other pro league—China),None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
per_game={}
for i in range(len(links_of_all)):
    per_game[names_of_all[i]]= get_stats(links_of_all[i])

In [15]:
per_game={k:v for (k,v) in per_game.items() if not v.empty}

In [21]:
adv={}
for i in range(len(links_of_all)):
    adv[names_of_all[i]]= get_advanced_stats(links_of_all[i])
adv={k:v for (k,v) in adv.items() if not v.empty}
for name in adv.keys():
    adv[name]['WS']= pd.to_numeric(adv[name]['WS'], errors='coerce')
    adv[name]['PER']= pd.to_numeric(adv[name]['PER'], errors='coerce')
    adv[name]['G']= pd.to_numeric(adv[name]['G'], errors='coerce')

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [19]:
for name in per_game.keys():
            per_game[name]= per_game[name][per_game[name]['Age']!='']

In [21]:
len(adv.keys())

2663

In [20]:
len(per_game.keys())

2651